# Optimizing Neural Network Hyperparameters

We will consider again an Image Recognition problem with the MNIST dataset (Images of 28 x 28 pixels). 
The MNIST dataset has a training set of 60,000 images and a test set of 10,000 images. 
The digits have been sized-normalized and centered in a fixed-size image.
It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting.

In this Notebook you will follow four different approaches to tune the hyperparameters:
- "Trial and Error" approach
- "Grid Search" Hyperparameter optimization with Scikit-Learn wrapper
- "Random Search" Hyperparameter optimization with Scikit-Learn wrapper
- Bayesian Optimization using Hyperas

### Data Preparation

In [ ]:
from keras.datasets import mnist
import numpy as np

(train_images, train_labels), (test_images, test_labels) = mnist.load_data() # train_x, train_y, test_x, test_y
# x values are the 28x28 pixels
# y values are the labels, the digit that is being represented in the image. We have 10 labels. Digits from 0 to 9.


In [ ]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

In [ ]:
# We need to encode the categorical labels in one-hot format

from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
# We split the training set into training and validation set
# 30% validation, 70% training

split_size = int(train_images.shape[0]*0.7)

train_images, val_images = train_images[:split_size], train_images[split_size:]
train_labels, val_labels = train_labels[:split_size], train_labels[split_size:]

In [ ]:
# Fix random seed for reproducibility
seed = 42
np.random.seed(seed)

### Building the Model

Build a neural network with 3 layers, input, hidden and output:
- Dense layer with 50 hidden units and an appropriate activation function;
- Dense layer with 10 output units and appropriate activation function;

First we define some useful parameters:

In [ ]:
# define vars
input_num_units = 784 # 28x28
hidden_num_units = 50
output_num_units = 10

epochs = 5
batch_size = 128

**Import Keras packages that you think may need.**

In [ ]:
# Import Keras modules
from keras import models
from keras import layers
from keras import optimizers

**Create the model as described above.**

In [ ]:
# Create a Sequential Model
model = models.Sequential()
model.add(layers.Dense(units=hidden_num_units, activation='relu', input_shape=(input_num_units,)))
model.add(layers.Dense(units=output_num_units, activation='softmax'))

# Print a summary of your model
model.summary()

In [ ]:
# or Create a functional model
from keras import Input
from keras import Model

inputs = Input(shape=(input_num_units,))
x = layers.Dense(hidden_num_units, activation='relu')(inputs)
outputs = layers.Dense(output_num_units, activation='softmax')(x)
func_model = Model(inputs=inputs, outputs=outputs)

# Print a summary of your model
# Here the input layer is shown in the summary
func_model.summary()

**Configure the model with an optimizer and an appropriate loss function.**

In [ ]:
# Compile the model with necessary attributes
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

**Train the model (also with the validation set).**

In [ ]:
# Train the model
trained_model = model.fit(train_images, train_labels, epochs=epochs, 
                          batch_size=batch_size, validation_data=(val_images, val_labels))

### Model Evaluation

**Test the model: get predictions and evaluate the model.**

In [ ]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

pred = np.argmax(model.predict(test_images), axis=-1)
label = np.argmax(test_labels, axis = 1)

confusion = confusion_matrix(label, pred)
print(confusion)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels) 

In [ ]:
print('test_acc:', test_acc)

Let's try to improve it by tuning some Hyperparameters.

### Hyperparameters Optimization - Trial and Error

Some important parameters to look out while optimizing neural networks are:
- Type of architecture
- Number of layers
- Number of neurons per layer
- Regularization parameters
- Learning rate
- Type of optimization/backpropagation technique
- Dropout rate
- Weight sharing

Now repeat all the previous steps (train, test etc..) but tuning the following parameters:
1. Make the model "wide": Increase the number of neurons in the hidden layer; 
2. Make the model "deep": Increase the number of hidden layers neurons each;
3. Dropout to deal with Overfitting;
4. Increase Epochs to 50;
5. Both "wide" and "deep": more hidden layers, each with more than 50 neurons

After every step, analyse your results and draw some conclusions.

### 1. Make the model "wide": increase number of neurons in the hidden layer.

**Define the new variables.**

In [ ]:
input_num_units = 784
hidden_num_units = 500
output_num_units = 10
epochs = 5
batch_size = 128

**Build the network.**

In [ ]:
model_1  = models.Sequential()
model_1.add(layers.Dense(units=hidden_num_units, activation='relu', input_shape=(input_num_units,)))
model_1.add(layers.Dense(units=output_num_units, activation='softmax'))

**Configure the network.**

In [ ]:
model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

**Train the network.**

In [ ]:
trained_model_1 = model_1.fit(train_images, train_labels, epochs=epochs, batch_size=batch_size, validation_data=(val_images, val_labels))

Considerations: what can you notice from these results? Is your model performing better than before?

**Results**: the model should perform significantly better than before.

### 2. Make the model "deep": Increase the number of hidden layers.

**Define the new variables.**

In [ ]:
input_num_units = 784
hidden1_num_units = 50
hidden2_num_units = 50
hidden3_num_units = 50
hidden4_num_units = 50
hidden5_num_units = 50
output_num_units = 10

epochs = 5
batch_size = 128

**Build the network.**

In [ ]:
model_2 = models.Sequential()

model_2.add(layers.Dense(units=hidden1_num_units, input_shape=(input_num_units,), activation='relu'))
model_2.add(layers.Dense(units=hidden2_num_units, activation='relu'))
model_2.add(layers.Dense(units=hidden3_num_units, activation='relu'))
model_2.add(layers.Dense(units=hidden4_num_units, activation='relu'))
model_2.add(layers.Dense(units=hidden5_num_units, activation='relu'))

model_2.add(layers.Dense(units=output_num_units, activation='softmax'))

model_2.summary()

**Configure the network.**

In [ ]:
model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

**Train the network.**

In [ ]:
trained_model_2 = model_2.fit(train_images, train_labels, epochs=epochs, 
                              batch_size=batch_size, validation_data=(val_images, val_labels))

Considerations: what can you notice from these results? Is your model performing better than before?

**Results**: the model is performing slightly worst than in the previous point. This may be due to a little bit of overfitting. To deal with this, we will use the dropout technique.

### 3. Dropout

**Define the new variables, remember to define also the dropout_ratio.**

In [ ]:
input_num_units = 784
hidden1_num_units = 50
hidden2_num_units = 50
hidden3_num_units = 50
hidden4_num_units = 50
hidden5_num_units = 50
output_num_units = 10

epochs = 5
batch_size = 128

dropout_ratio = 0.2

**Build the network.**

In [ ]:
model_3 = models.Sequential()

model_3.add(layers.Dense(units=hidden1_num_units, input_shape=(input_num_units,), activation='relu'))
model_3.add(layers.Dropout(dropout_ratio))
model_3.add(layers.Dense(units=hidden2_num_units, activation='relu'))
model_3.add(layers.Dropout(dropout_ratio))
model_3.add(layers.Dense(units=hidden3_num_units, activation='relu'))
model_3.add(layers.Dropout(dropout_ratio))
model_3.add(layers.Dense(units=hidden4_num_units, activation='relu'))
model_3.add(layers.Dropout(dropout_ratio))
model_3.add(layers.Dense(units=hidden5_num_units, activation='relu'))
model_3.add(layers.Dropout(dropout_ratio))

model_3.add(layers.Dense(output_num_units, activation='softmax'))

model_3.summary()

**Configure the network.**

In [ ]:
model_3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

**Train the network.**

In [ ]:
trained_model_3 = model_3.fit(train_images, train_labels, epochs=epochs, batch_size=batch_size, validation_data=(val_images, val_labels))

Considerations: what can you notice from these results? Is your model improving?

**Results**: There seems to be some problems, the model is not performing well enough. One possible reason may be that we are not using enough epochs to train the model. Let's try to increase the number of training epochs.

### 4. Increase training Epochs to 50.

This will take a while.

**Define the new variables.**

In [ ]:
input_num_units = 784
hidden1_num_units = 50
hidden2_num_units = 50
hidden3_num_units = 50
hidden4_num_units = 50
hidden5_num_units = 50
output_num_units = 10

epochs = 50
batch_size = 128

**Build the network.**

In [ ]:
model_4 = models.Sequential()

model_4.add(layers.Dense(units=hidden1_num_units, input_shape=(input_num_units, ), activation='relu'))
model_4.add(layers.Dropout(0.2))
model_4.add(layers.Dense(units=hidden2_num_units, activation='relu'))
model_4.add(layers.Dropout(0.2))
model_4.add(layers.Dense(units=hidden3_num_units, activation='relu'))
model_4.add(layers.Dropout(0.2))
model_4.add(layers.Dense(units=hidden4_num_units, activation='relu'))
model_4.add(layers.Dropout(0.2))
model_4.add(layers.Dense(units=hidden5_num_units, activation='relu'))
model_4.add(layers.Dropout(0.2))

model_4.add(layers.Dense(units=output_num_units, activation='softmax'))

model_4.summary()

**Configure the network.**

In [ ]:
model_4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

**Train the network.**

In [ ]:
trained_model_4 = model_4.fit(train_images, train_labels, epochs=epochs, 
                              batch_size=batch_size, validation_data=(val_images, val_labels))

Considerations: what can you notice from these results? Has the accuracy increased?

**Results**: Now seems better, there is an increase in the accuracy.

### 5. Make the model "wide" and "deep": more hidden layers, each with more than 50 neurons.

**Define the new variables.**

In [ ]:
input_num_units = 784
hidden1_num_units = 500
hidden2_num_units = 500
hidden3_num_units = 500
hidden4_num_units = 500
hidden5_num_units = 500
output_num_units = 10

epochs = 25
batch_size = 128

**Build the network.**

In [ ]:
model_5 = models.Sequential()

model_5.add(layers.Dense(units=hidden1_num_units, input_shape=(input_num_units,), activation='relu'))
model_5.add(layers.Dropout(0.2))
model_5.add(layers.Dense(units=hidden2_num_units,  activation='relu'))
model_5.add(layers.Dropout(0.2))
model_5.add(layers.Dense(units=hidden3_num_units,  activation='relu'))
model_5.add(layers.Dropout(0.2))
model_5.add(layers.Dense(units=hidden4_num_units,  activation='relu'))
model_5.add(layers.Dropout(0.2))
model_5.add(layers.Dense(units=hidden5_num_units, activation='relu'))
model_5.add(layers.Dropout(0.2))

model_5.add(layers.Dense(units=output_num_units,  activation='softmax'))

**Configure the network.**

In [ ]:
model_5.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

**Train the network. Use 25 epochs if 50 takes too long.**

In [ ]:
trained_model_5 = model_5.fit(train_images, train_labels, epochs=epochs, 
                              batch_size=batch_size, validation_data=(val_images, val_labels))

Considerations: what do you think about your final model?
It seems that we obtained our final solution model. Let's evaluate it with some predictions.

**Results**: Finally we get our very good model!

**Make some predictions and Evaluate the network.**

In [ ]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

pred = np.argmax(model_5.predict(test_images), axis=-1)
label = np.argmax(test_labels, axis = 1)

confusion = confusion_matrix(label, pred)
print(confusion)

In [ ]:
test_loss_final, test_acc_final = model_5.evaluate(test_images, test_labels) 

In [ ]:
print('test_acc:', test_acc_final)

### Hyperparameters Optimization - Grid Search

Instead of proceeding with a "trial and error" approach we can also use GridSearch to combine all the hyperparameters we want to tune. The Grid search works by trying every possible combination of parameters you want to try, therefore it will take a lot of time to perform the entire search.

What you have to do is to use Sequential() model in Keras as a part of the Scikit-Learn workflow via the wrappers. 
Check out how this [workflow](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html?highlight=gridsearch#sklearn.model_selection.GridSearchCV) works.


Please note that without GPU is extremely time consuming to tune all the hyperparameters in one shoot, by using an appropriate number of epochs. For this reason, in this example the idea is for you to understand how you can use GridSerach with Keras Model, but you will probably not be able to obtain an excellent model.

For this reason, try to tune the number of neurons in the hidden layers (more than one hidden layer) with just 5-10 epochs.

**Import GridSearchCV and KerasClassifier.**

In [ ]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm

In [ ]:
input_num_units = 784
output_num_units = 10

**Create a function called `create_model` in which you build your KerasClassifier with number of hidden units equal to a general variable (you can call this `neurons`). Inside the function you should then also compile the model.**

In [ ]:
# Function to create model, required for KerasClassifier
def create_model(neurons=1):
  
    model = models.Sequential()  
    model.add(layers.Dense(units=neurons, input_shape=(input_num_units,), activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(units=neurons, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(units=neurons, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(units=neurons, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(units=neurons, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(units=output_num_units, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

**Create a model wrapper.**

In [ ]:
model = KerasClassifier(build_fn=create_model, epochs=5, batch_size=32, verbose=1)

**Create a dictionary of parameters grid for the number of neurons in the hidden layer.**

In [ ]:
neurons = [50, 100, 250, 500]
param_grid = dict(neurons=neurons) 

**Grid Search: use `GridSearchCV`with the model you have obtained from the wrapper as estimator and the dictionary you have just created as param_grid.**

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=None)

**Fit the result from the Grid Search, call your result `grid_result`.**

In [ ]:
grid_result = grid.fit(train_images, train_labels, validation_data = (val_images, val_labels), epochs=5)

**Let's print some results. Fill in the #TO DOs with the best_score and best_params that you got after fitting.**

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) #TO DO, #TO DO
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    

### Hyperparameters Optimization - Random Search

Similarly, we can use Scikit-Learn workflow to perform [random search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html). In contrast to GridSearchCV, not all parameter values are tried out, but rather a fixed number of parameter settings is sampled from the specified distributions. The number of parameter settings that are tried is given by n_iter.


**Import RandomizedSearchCV.**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

**Random Search: use `RandomizedSearchCV` with the model you have obtained from the wrapper as estimator and the dictionary you have created for GridSearch as param_distributions.**

In [ ]:
# We reuse the estimator wrapper and the param_grid defined for GridSearch
print(param_grid)
random = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=2, n_jobs=None)

**Fit the result from the Grid Search, call your result `random_result`.**

In [ ]:
random_result = random.fit(train_images, train_labels, validation_data = (val_images, val_labels), epochs=1)

**Let's print some results. Fill in the #TO DOs with the best_score and best_params that you got after fitting.**

In [ ]:
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_)) #TO DO, #TO DO
means = random_result.cv_results_['mean_test_score']
stds = random_result.cv_results_['std_test_score']
params = random_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    

### Hyperparameters Optimization - Bayesian Optimization

Manual tuning (Trial and Error), grid search and random search are considered brute-force methods. 
Grid search is an exhaustive approach to evaluate the hyperparameter values defined in a grid. 
The random search approach will randomly choose the values for the hyperparameters. Grid and
random search perform better than manual tuning, but they do not consider past evaluations.
Hence, they lose significant time evaluating hyperparameters in the same inadequate region.
Furthermore, the objective functions to minimize are computationally expensive and running
them each time with a new set of parameters is time-demanding. 
Therefore, Bergstra et al. proposed a model-based method called Sequential Model-Based Optimization (SMBO).
The main idea of this approach is to create a surrogate model of the objective function using a
probabilistic distribution. This surrogate model bases on the Bayes’ theorem. At each new iteration, the surrogate will become more and more confident about which new guess can lead to improvements.

For this we will use the package [hyperas](https://github.com/maxpumperla/hyperas)

````pip install hyperas````

To do hyper-parameter optimization on a model, we just wrap the parameters we want to optimize into double curly brackets and choose a distribution. 

**The data for the optimizer should be returned from a function ````get_data````**

In [1]:
from keras.datasets import mnist
from keras.utils import to_categorical
import numpy as np


def get_data():
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data() 
    train_images = train_images.reshape((60000, 28 * 28))
    train_images = train_images.astype('float32') / 255

    test_images = test_images.reshape((10000, 28 * 28))
    test_images = test_images.astype('float32') / 255
    # We need to encode the categorical labels in one-hot format
    train_labels = to_categorical(train_labels)
    test_labels = to_categorical(test_labels)

    return train_images, train_labels, test_images, test_labels

Using TensorFlow backend.
C:\Users\sanchezloper\Anaconda3\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\sanchezloper\Anaconda3\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\sanchezloper\Anaconda3\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Us

**Define the create model function again, but returning not only the model but the function to minimize and the status**

In [2]:
from hyperas.distributions import choice
from hyperas import optim
from hyperopt import Trials, STATUS_OK, tpe 
# Import Keras modules
from keras import models
from keras import layers
from keras import optimizers

# Function to create model
def create_model():
  
    model = models.Sequential()  
    # Notice here how we define the number of neurons per layer
    model.add(layers.Dense(units={{choice([50, 100, 250, 500])}}, input_shape=(28*28,), activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(units={{choice([50, 100, 250, 500])}}, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(units={{choice([50, 100, 250, 500])}}, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(units={{choice([50, 100, 250, 500])}}, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(units={{choice([50, 100, 250, 500])}}, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(units=10, activation='softmax'))
    # model_compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # model fitting
    model.fit(train_images, train_labels, epochs=2, 
              batch_size=128, validation_split=0.3)

    score = model.evaluate(test_images, test_labels, verbose=0)
    accuracy = score[1]
    return {'loss': -accuracy,'status': STATUS_OK, 'model': model} #

**The last step is to run the optimization, which is done as follows:**

In [6]:
best_run, best_model = optim.minimize(model=create_model,
                                      data=get_data,
                                      algo=tpe.suggest,
                                      max_evals=5,
                                      trials=Trials(),
                                      notebook_name='OptimizingNN_Solutions')

>>> Imports:
#coding=utf-8

try:
    from keras.datasets import mnist
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from keras.utils import to_categorical
except:
    pass

try:
    from keras import models
except:
    pass

try:
    from keras import layers
except:
    pass

try:
    from keras import optimizers
except:
    pass

try:
    from keras import Input
except:
    pass

try:
    from keras import Model
except:
    pass

try:
    from sklearn.metrics import confusion_matrix, plot_confusion_matrix
except:
    pass

try:
    from sklearn.metrics import confusion_matrix, plot_confusion_matrix
except:
    pass

try:
    import numpy
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scikit_learn import KerasClassifier
except:
    pass

try:
    from keras.constraints import maxnorm
except:
    pass

try:
    from sklearn.model_selection import RandomizedSearchCV
except:
    pass

tr

                                                    
36224/42000 [========================>.....]         
 - ETA: 0s - loss: 0.5522 - accuracy: 0.8297         
                                                    
36736/42000 [=========================>....]         
 - ETA: 0s - loss: 0.5483 - accuracy: 0.8312         
                                                    
37120/42000 [=========================>....]         
 - ETA: 0s - loss: 0.5454 - accuracy: 0.8320         
                                                    
37504/42000 [=========================>....]         
 - ETA: 0s - loss: 0.5423 - accuracy: 0.8330         
                                                    
37888/42000 [==========================>...]         
 - ETA: 0s - loss: 0.5384 - accuracy: 0.8341         
                                                    
38272/42000 [==========================>...]         
 - ETA: 0s - loss: 0.5347 - accuracy: 0.8353         
                                  

                                                    
41728/42000 [============================>.]         
 - ETA: 0s - loss: 0.1879 - accuracy: 0.9487         
                                                    
42000/42000 [==============================]         
 - 5s 125us/step - loss: 0.1879 - accuracy: 0.9487 - val_loss: 0.1363 - val_accuracy: 0.9624

Train on 42000 samples, validate on 18000 samples                               
Epoch 1/2                                                                       
  128/42000 [..............................]                                    
 - ETA: 1:19 - loss: 2.2934 - accuracy: 0.1328                                  
                                                                               
  896/42000 [..............................]                                    
 - ETA: 13s - loss: 2.2748 - accuracy: 0.1417                                   
                                                                        

 - ETA: 3s - loss: 0.2663 - accuracy: 0.9211                                    
                                                                               
 1792/42000 [>.............................]                                    
 - ETA: 3s - loss: 0.2904 - accuracy: 0.9169                                    
                                                                               
 2432/42000 [>.............................]                                    
 - ETA: 3s - loss: 0.2844 - accuracy: 0.9194                                    
                                                                               
 3072/42000 [=>............................]                                    
 - ETA: 3s - loss: 0.2889 - accuracy: 0.9186                                    
                                                                               
 3712/42000 [=>............................]                                    
 - ETA: 3s - loss: 0.2828 - accu

 - ETA: 9s - loss: 1.8264 - accuracy: 0.3575                                    
                                                                               
 3584/42000 [=>............................]                                    
 - ETA: 8s - loss: 1.7505 - accuracy: 0.3856                                    
                                                                               
 3968/42000 [=>............................]                                    
 - ETA: 8s - loss: 1.6773 - accuracy: 0.4126                                    
                                                                               
 4352/42000 [==>...........................]                                    
 - ETA: 8s - loss: 1.6166 - accuracy: 0.4338                                    
                                                                               
 4736/42000 [==>...........................]                                    
 - ETA: 8s - loss: 1.5513 - accu

...

 - ETA: 3s - loss: 0.2722 - accuracy: 0.9222                                    
                                                                               
 3456/42000 [=>............................]                                    
 - ETA: 3s - loss: 0.2828 - accuracy: 0.9190                                    
                                                                               
 4096/42000 [=>............................]                                    
 - ETA: 3s - loss: 0.2727 - accuracy: 0.9221                                    
                                                                               
 4736/42000 [==>...........................]                                    
 - ETA: 3s - loss: 0.2767 - accuracy: 0.9215                                    
                                                                               
 5504/42000 [==>...........................]                                    
 - ETA: 2s - loss: 0.2826 - accu

 - ETA: 9s - loss: 1.8681 - accuracy: 0.3684                                    
                                                                               
 2816/42000 [=>............................]                                    
 - ETA: 9s - loss: 1.7654 - accuracy: 0.3991                                    
                                                                               
 3200/42000 [=>............................]                                    
 - ETA: 8s - loss: 1.6722 - accuracy: 0.4297                                    
                                                                               
 3584/42000 [=>............................]                                    
 - ETA: 8s - loss: 1.5999 - accuracy: 0.4562                                    
                                                                               
 3968/42000 [=>............................]                                    
 - ETA: 8s - loss: 1.5324 - accu

                                                                               
26112/42000 [=================>............]                                    
 - ETA: 2s - loss: 0.5566 - accuracy: 0.8240                                    
                                                                               
26368/42000 [=================>............]                                    
 - ETA: 2s - loss: 0.5532 - accuracy: 0.8251                                    
                                                                               
26624/42000 [==================>...........]                                    
 - ETA: 2s - loss: 0.5502 - accuracy: 0.8263                                    
                                                                               
26880/42000 [==================>...........]                                    
 - ETA: 2s - loss: 0.5474 - accuracy: 0.8274                                    
                                

 5120/42000 [==>...........................]                                    
 - ETA: 6s - loss: 0.1991 - accuracy: 0.9426                                    
                                                                               
 5376/42000 [==>...........................]                                    
 - ETA: 6s - loss: 0.1963 - accuracy: 0.9436                                    
                                                                               
 5760/42000 [===>..........................]                                    
 - ETA: 6s - loss: 0.1910 - accuracy: 0.9451                                    
                                                                               
 6144/42000 [===>..........................]                                    
 - ETA: 6s - loss: 0.1935 - accuracy: 0.9445                                    
                                                                               
 6528/42000 [===>...............

 - ETA: 2s - loss: 0.1720 - accuracy: 0.9510                                    
                                                                               
27392/42000 [==================>...........]                                    
 - ETA: 2s - loss: 0.1719 - accuracy: 0.9510                                    
                                                                               
27648/42000 [==================>...........]                                    
 - ETA: 2s - loss: 0.1721 - accuracy: 0.9512                                    
                                                                               
27904/42000 [==================>...........]                                    
 - ETA: 2s - loss: 0.1720 - accuracy: 0.9511                                    
                                                                               
28160/42000 [===================>..........]                                    
 - ETA: 2s - loss: 0.1726 - accu

**Let's print some results.**

In [8]:
train_images, train_labels, test_images, test_labels = get_data()
print("Evalutation of best performing model:")
print(best_model.evaluate(test_images, test_labels))

print("Best performing model chosen hyper-parameters:")
print(best_run)

Evalutation of best performing model:
10000/10000 [==============================] - 1s 66us/step
[0.12324347188323737, 0.9638000130653381]
Best performing model chosen hyper-parameters:
{'units': 3, 'units_1': 0, 'units_2': 2, 'units_3': 2, 'units_4': 1}
